# Analyze zoning districts

In [1]:
from sqlalchemy import create_engine
import geopandas as gpd

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

### Import districts

In [4]:
zoning = gpd.read_postgis("""SELECT * FROM parcelzoningpgh;""", DB, crs='EPSG:4326')

In [5]:
zoning['legendtype'] = zoning.legendtype.str.strip()
zoning = zoning[zoning.legendtype != 'Mount Oliver Borough']

zoning['legendtype'] = zoning.legendtype.replace('Single-Unit Attached Residential', 'Single-Unit A/D Residential')
zoning['legendtype'] = zoning.legendtype.replace('Single-Unit Detached Residential', 'Single-Unit A/D Residential')

### Remove rivers from districts

In [6]:
rivers = gpd.read_file('../input/rivers.geojson')

In [7]:
zoning = zoning.overlay(rivers, how='symmetric_difference', keep_geom_type=True)

### Summarize districts

In [8]:
agg = zoning.to_crs({ 'proj':'cea' })
agg['area'] = agg.area * 10.7639 # sq meters to sq feet
agg['area'] = agg.area / 43560 # sq feet to acres

In [9]:
agg = agg[~agg.legendtype.isin(['Parks', 'Hillside'])].groupby('legendtype').agg({ 'area': 'sum' }).reset_index()
agg['pct'] = agg.area / agg.area.sum()

In [10]:
agg.sort_values(by='pct', ascending=False)

,legendtype,area,pct
13,Single-Unit A/D Residential,1035.483662,0.446455
16,Two-Unit Residential,263.540607,0.113627
12,Riverfront,183.147037,0.078965
7,Multi-Unit Residential,176.764333,0.076213
20,Urban Industrial,138.404077,0.059674
11,Planned Unit Development,111.808882,0.048207
5,Local Neighborhood Commercial,90.898507,0.039191
0,Educational/Medical Institution,81.846908,0.035289
14,Specially Planned,50.288149,0.021682
1,General Industrial,36.992371,0.015949
